In [6]:
def domain(ds):
    
    """Returns the area of a lat/lon xarrray which us over land and within the bounds for this research"""
    
    # this is the mask of land area
    landMask=xarray.open_dataset(
        'helpers/cmip6LandMask.nc'
    )
    
    # apply a weighting by the percentage of the cell which is land area
    # and set non-land area to Nan
    result=(ds*landMask.sftlf/100).where(
        landMask.sftlf>0
    )
    
    #slice to my area of interest
    #I added a 0.75 margin as this is half the grid square size
    result=result.sel(
        lat=slice(-39-0.75,-28+0.75),
        lon=slice(144-0.75,155+0.75)
    )
    
    if type(ds)==xarray.core.dataarray.DataArray:
        #keep the old name
        result=result.rename(ds.name)

    return result

In [7]:
def emergePlot(binnedXr, yLabel='Events per 30 Years', xlim=[1900,2100]):
    '''
    plot indeces for multi-experiment mean, and add some lines for CI

    '''
    #somewhere to save axes
    ax=list()
    
    # get a list of the coords, and remove year. Which leaves experiment or model.
    dims=list(binnedXr.coords)
    dims.remove('year') 
    
    #Z = 1.645 #Z-score for a 90% CI
    Z=1.96 #Z-score for a 95% CI

    #these are the variable/index names to plot
    #all variables that not empty
    indexToPlot = list(binnedXr.where(binnedXr.isnull().all()==False).data_vars)
    nIndices=len(indexToPlot)

    # define somewhere to plot
    fig = plt.figure(figsize=(12,2*nIndices))

    # for each variable number
    for i in range(nIndices):

        #create a subplot (row, column, size)
        ax.append(fig.add_subplot(nIndices,1,i+1))

        # plot the actual data
        plotData=binnedXr[indexToPlot[i]]

        mean=plotData.mean(dims)
        std=plotData.std(dims)
        
        #blue line for multi-experiment mean, and light blue shading between min and max
        plt.plot(
            mean.year, 
            mean, 
            label=indexToPlot[i]
        )
        plt.fill_between(
            mean.year, 
            #mean+Z*std,
            #mean-Z*std,
            plotData.min(dims),
            plotData.max(dims),
            color='lightblue'
        )

        #Tidy up a bit
        plt.xlim(xlim)
        plt.ylabel(yLabel)
        plt.legend()

    return ax

In [8]:
def emergePlotWithModelCount(binnedXr, climatXr, yLabel='Events per 30 Years', xlim=[1900,2100]):
    '''
    plot indeces for multi-experiment mean, and add some lines for CI

    '''
    #somewhere to save axes
    ax=list()
    
    # multi-experiment mean state (by model and time)
    meanXr=climatXr.mean('year')
    stdXr=climatXr.std('year')

    # get a list of the coords, and remove year. Which leaves experiment or model.
    dims=list(climatXr.coords)
    dims.remove('year') 
    
    #Z = 1.645 #Z-score for a 90% CI
    Z=1.96 #Z-score for a 95% CI

    #these are the variable/index names to plot
    #all variables that not empty
    indexToPlot = list(binnedXr.where(binnedXr.isnull().all()==False).data_vars)
    nIndices=len(indexToPlot)

    #emergence
    emergeXr=binnedXr>(meanXr+Z*stdXr)
    
    # define somewhere to plot
    fig = plt.figure(figsize=(12,4*nIndices))

    # for each variable number
    for i in range(nIndices):

        #create a subplot (row, column, size)
        ax.append(fig.add_subplot(2*nIndices,1,2*i+1))

        # plot the actual data
        plotData=binnedXr[indexToPlot[i]]

        
        mmmean=plotData.mean(dims)
        
        #blue line for multi-experiment mean, and light blue shading between min and max
        plt.plot(mmmean.year, 
                 mmmean, 
                 label=indexToPlot[i])
        plt.fill_between(mmmean.year, 
                         plotData.min(dims),
                         plotData.max(dims),
                         color='lightblue')
        #Tidy up a bit
        plt.xlim(xlim)
        plt.ylabel(yLabel)
        plt.legend()
        
        #create a subplot (row, column, size)
        ax.append(fig.add_subplot(2*nIndices,1,2*i+2))
        
        emergeNd=emergeXr[indexToPlot[i]].sel(year=slice(1865,2084),drop=True).values
        
        posEmergeNd=numpy.greater(
            emergeNd[:,1:-1],
            emergeNd[:,0:-2]
        )
        negEmergeNd=numpy.less(
            emergeNd[:,1:-1],
            emergeNd[:,0:-2]
        )
        #print((posEmergeNd))
        #print((negEmergeNd))
        modelCount=numpy.ndarray(220)
        modelCount[0]=0
        for iYear in range(1,220):
            modelCount[iYear]=(
                numpy.count_nonzero(
                    (numpy.all(posEmergeNd[
                        :,
                        iYear:]==False,
                               1
                              ))
                    &numpy.not_equal(
                        numpy.count_nonzero(posEmergeNd,1
                                           ),
                        numpy.count_nonzero(negEmergeNd,1
                                           )-emergeNd[:,
                            0]
                    )
                 )
            )
            
        print(modelCount[-1])
        
        plt.fill_between(
            range(0,220),
            0,
            modelCount/35*100,
            color='brown', 
            alpha=0.5
        )
        
        plt.hlines(70,0,2100, color='grey', linestyle='dashed')
        
        plt.xticks(ticks=[35,60,85,110,135,160,185,210], labels=range(1900,2100,25))
        plt.yticks([0,35,70,100])
        plt.ylim([0,105])
        plt.xlim((xlim[0]-1865,xlim[1]-1865))
        plt.ylabel('Emerge (%)')

        

    return ax

In [9]:
def modelEmergePlot(binnedXr, climatXr, yLabel='Events per 30 Years', xlim=[1900,2100]):

    ax=list()
    # plot indeces for multi-experiment mean, and add some lines for CI
    
    # multi-experiment mean state (by index and time)
    meanXr=climatXr.mean('year')
    
    dims=list(climatXr.coords)
    dims.remove('year') #this is normall experiment (for CESM) or model (for CMIP)
    
    stdXr=climatXr.std('year')
    
    #Z = 1.645 #Z-score for a 90% CI
    Z=1.96 #Z-score for a 95% CI

    #these are the variable/index names to plot
    #all variables that not empty
    indexToPlot = list(binnedXr.where(binnedXr.isnull().all()==False).data_vars)
    
    nIndices=len(indexToPlot)

    # define somewhere to plot
    fig = plt.figure(figsize=(12,2*nIndices))

    # for each variable number
    for i in range(nIndices):

        #create a subplot (row, column, size)
        ax.append(fig.add_subplot(nIndices,1,i+1))

        # plot the actual data
        plotData=binnedXr[indexToPlot[i]]

        
        mean=plotData.mean(dims)
        std=plotData.std(dims)
        
        #blue line for multi-experiment mean, and light blue shading between min and max
        plt.plot(mean.year, 
                 mean, 
                 label=indexToPlot[i])
        plt.fill_between(mean.year, 
                         #mean+Z*std,
                         #mean-Z*std,
                         plotData.min(dims),
                         plotData.max(dims),
                         color='lightblue')


        # draw some horizontal lines for criteria
        plt.axhline(y=meanXr[indexToPlot[i]]+Z*stdXr[indexToPlot[i]], dashes=(5,5), color='black')
        plt.axhline(y=meanXr[indexToPlot[i]]-Z*stdXr[indexToPlot[i]], dashes=(5,5), color='black')

        #Tidy up a bit
        plt.xlim(xlim)
        plt.ylabel(yLabel)
        plt.legend()

    return ax

In [10]:
def overlaidEmergePlot(binnedXr, *args):

    # plot indeces for multi-experiment mean, and add some lines for CI
    
   
    
    dims=list(binnedXr.dims)
    dims.remove('year') #this is normall experiment (for CESM) or model (for CMIP)
    
    if len(args)==1:

        climatXr=args[0]
    
        # multi-experiment mean state (by index and time)
        meanXr=climatXr.mean()

        # multi experiment std
        stdXr=climatXr.mean(dims).std()
    

    #Z = 1.645 #Z-score for a 90% CI
    Z=1.96 #Z-score for a 95% CI

    #these are the variable/index names to plot
    #all variables that not empty
    indexToPlot = list(binnedXr.where(binnedXr.isnull().all()==False).data_vars)
    
    nIndices=len(indexToPlot)

    # define somewhere to plot
    #fig = plt.figure(figsize=(10,5))
    #fig.add_subplot(1,1,1)

    
    mainColor=['red','blue','green','orange','purple']
    
    # for each variable number
    for i in range(nIndices):

        # plot the actual data
        plotData=binnedXr[indexToPlot[i]]

        #blue line for multi-experiment mean, and light blue shading between min and max
        plt.plot(
            plotData.year.values, 
            plotData.mean(dims).values, 
            label=indexToPlot[i], 
            color=mainColor[i], 
        )
        plt.fill_between(
            plotData.year.values, 
            plotData.min(dims).values, 
            plotData.max(dims).values, 
            color=mainColor[i], 
            alpha=0.1
        )

        if len(args)==1:
            # draw some horizontal lines for criteria
            plt.axhline(y=meanXr[indexToPlot[i]]+Z*stdXr[indexToPlot[i]], dashes=(5,5), color=mainColor[i])
            plt.axhline(y=meanXr[indexToPlot[i]]-Z*stdXr[indexToPlot[i]], dashes=(5,5), color=mainColor[i])

        #plt.axvline(x=1995)
        
        #Tidy up a bit
        plt.xlabel('Year')
        plt.ylabel('Number of Events/30 Years')
        plt.minorticks_on()
        #plt.legend()
        #plt.grid(axis='y', color='grey', dashes=[6,3])

